# Notebook to test whether D. Reichs big Eigenstrat File can be loaded

In [1]:
import numpy as np
import pandas as pd
import os as os
import sys as sys
import multiprocessing as mp
import socket

### Pick the right path (whether on cluster or at home)
socket_name = socket.gethostname()
print(socket_name)
if socket_name == "VioletQueen":
    path = "/home/harald/git/HAPSBURG/"   # The Path on Harald's machine
elif socket_name.startswith("midway2"):
    print("Midway jnovmbre partition detected.")
    path = "/project/jnovembre/hringbauer/HAPSBURG/"  # The Path on Midway Cluster
else: 
    raise RuntimeWarning("Not compatible machine. Check!!")
    
os.chdir(path)  # Set the right Path (in line with Atom default)

sys.path.append("./Python3/")  # Since now we are in the Root Directory
from hmm_inference import HMM_Analyze   # Do not move. Should be after sys.path..
#sys.path.append("./Python3/create1000G_Mosaic/")  # Since now we are in the Root Directory
#from createMosaicsMulti import Mosaic_1000G_Multi  # Import the object that can create the Multiruns

print(os.getcwd()) # Show the current working directory. Should be HAPSBURG/Notebooks/ParallelRuns
print(f"CPU Count: {mp.cpu_count()}")

VioletQueen
/home/harald/git/HAPSBURG
CPU Count: 4


### The BaseBath (without the .snp/ind/geno)

In [51]:
folder = "./Data/ReichLabEigenstrat/Olalde2019/Olalde_et_al_genotypes"

### Load the SNP File

In [56]:
%%time
# On Harald's machine: About 7.14s
rsnp = np.loadtxt(folder + ".snp", dtype="str") 
print(f"Loaded {len(rsnp)} SNPs")

Loaded 1233013 SNPs
CPU times: user 6.72 s, sys: 537 ms, total: 7.26 s
Wall time: 7.28 s


## Load the Geno File

In [57]:
%%time
geno_file = open(folder+".geno", "rb")
header=geno_file.read(20)         #Ignoring hashes
#if header.split()[0]!="GENO":
#    raise Exception("This does not look like a packedancestrymap file")
nind,nsnp=[int(x) for x in header.split()[1:3]]        
nind=nind
nsnp=nsnp
rlen=max(48,int(np.ceil(nind*2/8)))    #assuming sizeof(char)=1 here
geno_file.seek(rlen)         #set pointer to start of genotypes

geno=np.fromfile(folder+".geno", dtype='uint8')[rlen:] #without header
geno.shape=(nsnp, rlen)
geno=np.unpackbits(geno, axis=1)[:,:(2*nind)]
geno=2*geno[:,::2]+geno[:,1::2]
#geno=geno[:,:]
geno[geno==3]=9                       #set missing values
geno = geno[:,0]

print(nind)
print(nsnp)

278
1233013
CPU times: user 2.15 s, sys: 483 ms, total: 2.64 s
Wall time: 2.64 s


## Run it piece by piece
(for debugging)

In [58]:
%%time
geno_file = open(folder+".geno", "rb")
header=geno_file.read(20)         #Ignoring hashes
nind,nsnp=[int(x) for x in header.split()[1:3]]        
nind=nind
nsnp=nsnp
rlen=max(48,int(np.ceil(nind*2/8)))    #assuming sizeof(char)=1 here

CPU times: user 1.18 ms, sys: 0 ns, total: 1.18 ms
Wall time: 1.21 ms


In [59]:
nind

278

In [60]:
rlen

70

In [61]:
%%time
geno=np.fromfile(folder+".geno", dtype='uint8')[rlen:] #without header
geno.shape=(nsnp, rlen)
print(geno.shape)

(1233013, 70)
CPU times: user 0 ns, sys: 148 ms, total: 148 ms
Wall time: 151 ms


### Unpack only the first couple of individuals
(set by n unload)

In [62]:
n_sub =8
rlen_sub = int(np.ceil(n_sub*2/8))
print(rlen_sub)

2


In [63]:
geno_sub = geno[:,:rlen_sub]
geno_sub=np.unpackbits(geno_sub, axis=1)[:,:(2*n_sub)]
geno_sub=2*geno_sub[:,::2]+geno_sub[:,1::2]
#geno=geno[:,:]
geno_sub[geno_sub==3]=9                       #set missing values
#geno_sub = geno_sub[:,0]

print(np.shape(geno_sub))

(1233013, 8)


In [64]:
np.sum(geno_sub[:,6]!=9)

1051699

# And now write own class to extract specific Individual

In [39]:
class EigenstratLoad(object):
    """Class that loads and postprocesses Eigenstrats"""
    base_path = "./Data/ReichLabEigenstrat/Raw/v37.2.1240K"
    nsnp = 0
    nind = 0
    rlen = 0
    
    def __init__(self, base_path=""):
        if len(base_path)==0:
            base_path = self.base_path # Go to Default
        geno_file = open(base_path +".geno", "rb")
        header = geno_file.read(20)         #Ignoring hashes
        self.nind, self.nsnp = [int(x) for x in header.split()[1:3]]        
        self.rlen=max(48, int(np.ceil(nind*2/8)))    #assuming sizeof(char)=1 here
        print(f"File with {self.nind} Individuals and {self.nsnp} SNPs")
                       
    def get_geno_i(self, i, missing_val=3):
        """Load Individual i"""
        batch, eff = self.get_enc_index(i)
        geno = self.give_bit_file()  # Load the whole bit file

        geno_sub = geno[:,[batch]]  # Byte value of batch
        geno_sub = np.unpackbits(geno_sub, axis=1)[:,2*eff:2*eff+2]
        geno_sub = 2*geno_sub[:,0] + geno_sub[:,1]
        geno_sub[geno_sub==3] = missing_val  #set missing values
        return geno_sub
      
    def get_geno_indvidiaul_iid(self, iid):
        """Return Genotypes of Individual iid"""
        i = self.get_i_from_iid(iid)
        g = self.get_geno_i(i)
        return g
        
    def get_i_from_iid(self, iid):
        """Get the Index of Individual iid"""
        raise NotImplementedError("Implement this")
        
    def give_bit_file(self):
        base_path = self.base_path
        geno=np.fromfile(base_path+".geno", dtype='uint8')[self.rlen:] #without header
        geno.shape=(self.nsnp, self.rlen)
        return geno
    
    def get_enc_index(self, i):
        """Get the Index in the Encoding and the modulo 4 value
        (position in batch)"""
        rlen_sub = int(np.floor(i * 2/8)) # Effectively dividing by four
        mod_i = i % 4  # Calculate the rest
        return rlen_sub, mod_i

### Test the Code

In [63]:
%%time
es = EigenstratLoad(base_path = "./Data/ReichLabEigenstrat/Raw/v37.2.1240K")
g = es.get_geno_i(0)

File with 5081 Individuals and 1233013 SNPs
CPU times: user 31.7 ms, sys: 848 ms, total: 880 ms
Wall time: 887 ms


In [80]:
%%time
es = EigenstratLoad(base_path = "./Data/ReichLabEigenstrat/Raw/v37.2.1240K")
for i in range(10):
    g = es.get_geno_i(i)
    print(f"Avg. Coverage: {1-np.mean(g==3):.6f}")
    print(f"SNps covered: {np.sum(g!=3)}")

File with 5081 Individuals and 1233013 SNPs
Avg. Coverage: 0.048705
SNps covered: 60054
Avg. Coverage: 0.105648
SNps covered: 130265
Avg. Coverage: 0.112596
SNps covered: 138832
Avg. Coverage: 0.056045
SNps covered: 69104
Avg. Coverage: 0.435407
SNps covered: 536862
Avg. Coverage: 0.055553
SNps covered: 68498
Avg. Coverage: 0.631162
SNps covered: 778231
Avg. Coverage: 0.026451
SNps covered: 32614
Avg. Coverage: 0.091817
SNps covered: 113211
Avg. Coverage: 0.055737
SNps covered: 68724
CPU times: user 364 ms, sys: 7.19 s, total: 7.55 s
Wall time: 7.55 s


# Test the Python file

In [2]:
sys.path.append("./PackagesSupport/loadEigenstrat/")  # Since now we are in the Root Directory
from loadEigenstrat import EigenstratLoad

In [3]:
es = EigenstratLoad(base_path = "./Data/ReichLabEigenstrat/Raw/v37.2.1240K")

File with 5081 Individuals and 1233013 SNPs


In [4]:
es.df_snp

array([['rs3094315', '1', '0.020130', '752566', 'G', 'A'],
       ['rs12124819', '1', '0.020242', '776546', 'A', 'G'],
       ['rs28765502', '1', '0.022137', '832918', 'T', 'C'],
       ...,
       ['rs28628009', '24', '0', '59033110', 'A', 'T'],
       ['rs55686319', '24', '0', '59033139', 'T', 'C'],
       ['rs75089321', '24', '0', '59033249', 'G', 'T']], dtype='<U15')

In [42]:
for i in range(10):
    g = es.get_geno_i(i)
    print(f"Avg. Coverage: {1-np.mean(g==3):.6f}")
    print(f"SNps covered: {np.sum(g!=3)}")

Avg. Coverage: 0.048705
SNps covered: 60054
Avg. Coverage: 0.105648
SNps covered: 130265
Avg. Coverage: 0.112596
SNps covered: 138832
Avg. Coverage: 0.056045
SNps covered: 69104
Avg. Coverage: 0.435407
SNps covered: 536862
Avg. Coverage: 0.055553
SNps covered: 68498
Avg. Coverage: 0.631162
SNps covered: 778231
Avg. Coverage: 0.026451
SNps covered: 32614
Avg. Coverage: 0.091817
SNps covered: 113211
Avg. Coverage: 0.055737
SNps covered: 68724


In [28]:
g = es.get_geno_i(953)
print(f"Avg. Coverage: {1-np.mean(g==3):.6f}")
print(f"SNps covered: {np.sum(g!=3)}")

Avg. Coverage: 0.921500
SNps covered: 1136222


### And now try out the Olalde Paper

In [5]:
es = EigenstratLoad(base_path = "./Data/ReichLabEigenstrat/Olalde2019/Olalde_et_al_genotypes")

File with 278 Individuals and 1233013 SNPs


In [6]:
for i in range(10):
    g = es.get_geno_i(i)
    print(f"Avg. Coverage: {1-np.mean(g==3):.6f}")
    print(f"SNps covered: {np.sum(g!=3)}")

Avg. Coverage: 0.122928
SNps covered: 151572
Avg. Coverage: 0.436765
SNps covered: 538537
Avg. Coverage: 0.047256
SNps covered: 58267
Avg. Coverage: 0.576944
SNps covered: 711380
Avg. Coverage: 0.120054
SNps covered: 148028
Avg. Coverage: 0.347864
SNps covered: 428921
Avg. Coverage: 0.852950
SNps covered: 1051699
Avg. Coverage: 0.098032
SNps covered: 120875
Avg. Coverage: 0.517977
SNps covered: 638672
Avg. Coverage: 0.203932
SNps covered: 251451


# Area 51

In [49]:
%%time
np.loadtxt("./Data/ReichLabEigenstrat/Olalde2019/Olalde_et_al_genotypes.snp", dtype="str")

CPU times: user 6.68 s, sys: 180 ms, total: 6.86 s
Wall time: 6.93 s


array([['rs3094315', '1', '0.020130', '752566', 'G', 'A'],
       ['rs12124819', '1', '0.020242', '776546', 'A', 'G'],
       ['rs28765502', '1', '0.022137', '832918', 'T', 'C'],
       ...,
       ['rs28628009', '24', '0', '59033110', 'A', 'T'],
       ['rs55686319', '24', '0', '59033139', 'T', 'C'],
       ['rs75089321', '24', '0', '59033249', 'G', 'T']], dtype='<U15')

In [14]:
%%time
df_snp = pd.read_csv("./Data/ReichLabEigenstrat/Olalde2019/Olalde_et_al_genotypes.snp", header=None, 
                     sep=r"\s+", engine="python")
df_snp.columns = ["SNP", "chr", "map", "pos", "ref", "alt"] # Set the Columns

CPU times: user 9.65 s, sys: 131 ms, total: 9.78 s
Wall time: 9.79 s


In [20]:
ch =1
df_snp[df_snp["chr"]==ch]

SNP  chr       map        pos ref alt
0        rs3094315    1  0.020130     752566   G   A
1       rs12124819    1  0.020242     776546   A   G
2       rs28765502    1  0.022137     832918   T   C
3        rs7419119    1  0.022518     842013   T   G
4         rs950122    1  0.022720     846864   G   C
5      rs113171913    1  0.023436     869303   C   T
6       rs13302957    1  0.024116     891021   G   A
7       rs59986066    1  0.024183     893462   C   T
8      rs112905931    1  0.024260     896271   C   T
9        rs6696609    1  0.024457     903426   C   T
10      rs13303368    1  0.024771     914852   G   C
11          rs8997    1  0.025727     949654   A   G
12       rs4075116    1  0.026208    1003629   C   T
13       rs3934834    1  0.026219    1005806   C   T
14       rs9442372    1  0.026288    1018704   A   G
15       rs3737728    1  0.026324    1021415   A   G
16      rs11260588    1  0.026330    1021658   G   A
17       rs9442398    1  0.026331    1021695   A   G
18       rs6687776    1  0.026621    1030565   C   T
19       rs9651273    1  0.026628    1031540   A   G
20     rs147606383    1  0.026665    1045331   G   A
21       rs4970405    1  0.026674    1048955   A   G
22      rs12726255    1  0.026677    1049950   A   G
23       rs7540009    1  0.026704    1060235   G   A
24      rs11807848    1  0.026711    1061166   T   C
25       rs9442373    1  0.026733    1062638   C   A
26       rs2298217    1  0.026806    1064979   C   T
27      rs12145826    1  0.026839    1066029   G   A
28       rs4970357    1  0.027073    1077064   C   A
29      rs11260603    1  0.027116    1079198   T   C
...            ...  ...       ...        ...  ..  ..
93136    rs7532941    1  2.861465  249066372   T   C
93137   rs12748771    1  2.861524  249068929   T   C
93138   rs12750069    1  2.861533  249069190   G   A
93139   rs12563983    1  2.861582  249074032   T   C
93140    rs6683472    1  2.861588  249075377   T   G
93141   rs11205395    1  2.861600  249076621   C   T
93142    rs5024942    1  2.861620  249077321   T   C
93143   rs11205405    1  2.861741  249081043   G   A
93144    rs4281354    1  2.861773  249083236   G   A
93145    rs7529822    1  2.861832  249088459   G   A
93146    rs4539172    1  2.861835  249089971   G   T
93147   rs12059437    1  2.861863  249094557   C   T
93148    rs4307600    1  2.861945  249105163   T   C
93149    rs6688948    1  2.861990  249114846   C   T
93150    rs4926509    1  2.861993  249115467   C   A
93151    rs7526416    1  2.862003  249117341   G   A
93152   rs11205415    1  2.862020  249120684   T   G
93153   rs11800006    1  2.862039  249122243   T   C
93154    rs9727917    1  2.862095  249126973   A   G
93155   rs10157709    1  2.862202  249154567   T   C
93156    rs4072273    1  2.862210  249158239   A   C
93157    rs6587759    1  2.862221  249163530   G   A
93158   rs12033802    1  2.862235  249164875   T   C
93159   rs10788875    1  2.862322  249170711   G   T
93160    rs4359061    1  2.862432  249174682   G   A
93161   rs12043282    1  2.862505  249179856   T   C
93162   rs12087178    1  2.862563  249188627   G   A
93163    rs6587762    1  2.862626  249198164   G   A
93164    rs7527472    1  2.862668  249202567   C   T
93165    rs6704311    1  2.862730  249210707   G   A

[93166 rows x 6 columns]

In [41]:
df_t = df_snp.loc[df_snp["chr"]==1, ["ref", "alt"]]

In [42]:
ref, alt = df_t["ref"].values, df_t["alt"].values

In [44]:
alt

array(['A', 'G', 'C', ..., 'A', 'T', 'A'], dtype=object)

In [48]:
pos = df_snp.loc[df_snp["chr"]==ch, "pos"]
pos.values

array([   752566,    776546,    832918, ..., 249198164, 249202567,
       249210707])

In [51]:
%%time
df_ind = pd.read_csv("./Data/ReichLabEigenstrat/Olalde2019/Olalde_et_al_genotypes.ind", header=None, 
                     sep=r"\s+", engine="python")
df_ind.columns = ["iid", "sex", "cls"] # Set the Columns

CPU times: user 2.38 ms, sys: 3.95 ms, total: 6.33 ms
Wall time: 5.9 ms


In [53]:
len(df_ind)

278

In [55]:
df_ind.head(5)

iid sex                  cls
0  I3807   M  SE_Iberia_c.10-16CE
1  I3981   M    SE_Iberia_c.5-8CE
2  I4055   M    SE_Iberia_c.3-4CE
3  I4246   M      C_Iberia_CA_Afr
4  I2472   M          N_Iberia_BA

In [61]:
np.where(df_ind["iid"]=='I4055')[0][0]

2